## Intro 4: saving, reloading, checkpointing

Basic checkpointing is acheived by converting TectonicModel objects to dictionary, then pickling. 

UWSubduction includes a small class called checkpoint that helps organise the checkponting of underworld2 objects and Python dictionaries.

The dictionary only stores the underlying Networkx graph, containing information about plates/boundaries and velocities. This means we always need to rebuild the TectonicModel object every time the Uw model is run. The saved data gets laoded by using the `pop_from_dict_of_lists` method.

In [40]:
#use this block to point to a local version of UWsubduction
import sys

try:
    sys.path.append('../..')
except:
    pass

In [39]:
import numpy as np
import os
import pickle
import underworld as uw
from underworld import function as fn
import glucifer
from easydict import EasyDict as edict
import networkx as nx
import operator
import UWsubduction as usub

In [38]:
from UWsubduction.utils import checkpoint

#create a checkpoint object
cp = checkpoint('./checkpoint/')

startFresh = True #if this false we'll load the TectonicModel from a checkpoint (a pickled dictionary)

In [20]:
#if restart, attempt to read in saved dicts. 
if cp.restart:
    with open(os.path.join(cp.loadpath, 'tmDict.pkl'), 'rb') as fp:
                            tmDict = pickle.load(fp)
            
tmDict.keys() 

[1, 2, 3]

## A toy model

In [21]:


mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1"), 
                                 elementRes  = (96, 48), 
                                 minCoord    = (-1.0, 0.0), 
                                 maxCoord    = (1.0, 1.0) )


temperatureField = uw.mesh.MeshVariable( mesh=mesh, nodeDofCount=1 )
velocityField = uw.mesh.MeshVariable( mesh=mesh, nodeDofCount=2 )
coordinate = fn.input()


In [27]:
#toy model, 3 plates


tm = usub.TectonicModel(mesh, 0, 20, 1)

if not cp.restart or startFresh is True:
    print('starting fresh')
    
    tm.add_plate(1, velocities = 1)
    tm.add_plate(2,velocities = -1)
    tm.add_plate(3, velocities = 0)

    tm.add_left_boundary(1)
    tm.add_subzone(1, 2, 0.25, subInitAge=50.0, upperInitAge=10.0)
    tm.add_ridge(2, 3, 0.5)
    tm.add_right_boundary(3, 30.)
    
else:
    print('loading from checkpoint / dict')
    tm.pop_from_dict_of_lists(tmDict)

starting fresh


In [28]:
#tm.is_subduction_boundary((1,2))

## Checkpointing

In [29]:
tmDict = nx.to_dict_of_dicts(tm)
cp.addDict(tmDict, 'tmDict')

## Check updatedness of tmDict whane we change tm

In [30]:
print(tm[2][3]['loc'])
print(tmDict[2][3]['loc'])

0.5
0.5


In [31]:
tm.set_bound_loc((2,3), 0.6)
print(tm[2][3]['loc'])
print(tmDict[2][3]['loc'])

0.6
0.6


In [32]:
print(tmDict[2][3]['velocities'])

[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan]


In [33]:
tm[2][3]['velocities'] = np.ones(len(tm[2][3]['velocities'] ))

In [34]:
print(tmDict[2][3]['velocities'])

[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.]


# Checkpointing the dict

In [35]:
step = 0
time = 0.0

cp.saveDicts(step, time)